In [2]:
import tools
import visualization_2D as vis2D
import visualization_3D as vis3D
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.graphics.gofplots import qqplot_2samples
from scipy.stats import ks_2samp
from scipy.stats import chi2_contingency
import matplotlib.pyplot as plt
import random as rd
import plotly.graph_objects as go
import plotly.express as px
import os
import sqlite3
from intermine.webservice import Service
import matplotlib.pyplot as plt

%matplotlib inline

# I- TRN download

In [ ]:
%%time
service = Service("https://yeastmine.yeastgenome.org/yeastmine/service")
query = service.new_query("Gene")
query.add_constraint("regulatoryRegions", "TFBindingSite")
query.add_view(
    "regulatoryRegions.regulator.symbol",
    "regulatoryRegions.regulator.secondaryIdentifier", "symbol",
    "secondaryIdentifier", "regulatoryRegions.regEvidence.ontologyTerm.name",
    "regulatoryRegions.regEvidence.ontologyTerm.identifier",
    "regulatoryRegions.experimentCondition",
    "regulatoryRegions.strainBackground",
    "regulatoryRegions.regulationDirection",
    "regulatoryRegions.publications.pubMedId", "regulatoryRegions.datasource",
    "regulatoryRegions.annotationType"
)
query.add_constraint("regulatoryRegions.regulator", "IN", "ALL_Verified_Uncharacterized_Dubious_ORFs", code="A")
query.add_constraint("regulatoryRegions.strainBackground", "=", "S288c", code="B")
query.add_constraint("regulatoryRegions.regEvidence.ontologyTerm.name", "ONE OF", 
                     ["chromatin immunoprecipitation-chip evidence", 
                      "chromatin immunoprecipitation- exonuclease evidence", 
                      "northern blot evidence", 
                      "western blot evidence", 
                      "chromatin immunoprecipitation-qPCR evidence", 
                      "beta-galactosidase reporter gene assay evidence", 
                      "chromatin immunoprecipitation-PCR evidence", 
                      "co-immunoprecipitation evidence", 
                      "chromatin immunoprecipitation evidence", 
                      "protein expression level evidence based on western blot"], code="C")

# Uncomment and edit the code below to specify your own custom logic:
# query.set_logic("B and A and C and C")

In [ ]:
%%time
TRN = pd.DataFrame(columns=['TF', 'TG'])

for row in query.rows():
    TRN = TRN.append({'TF': row["regulatoryRegions.regulator.secondaryIdentifier"],
                      'TG': row["secondaryIdentifier"]}, ignore_index=True)

display(TRN)

# II- Targets lists creation

In [ ]:
list_TF = TRN["TF"].unique()
print(f"Number of TF: {len(list_TF)}")

for TF in list_TF :
    TG = TRN[TRN["TF"] == TF]["TG"]
    TG.to_csv(f"../results/V2/TF_target_TRN/{TF}_{len(TG)}_targets.csv",
              index = False)

# III- Cumulative distribution function and distribution histogram

## 1) For 3D distances between targets

In [ ]:
def get_edges_list(gene_list, edges_list, feature_name):
    
    # Add SGDID
    feature_name = feature_name.merge(genes_list, left_on = "Feature_name", right_on = genes_list.columns[0])
    
    # Extract distances for selected genes list
    edges_list_select = edges_list[edges_list["Primary_SGDID"].isin(feature_name["Primary_SGDID"])]
    edges_list_select = edges_list_select[edges_list_select["Primary_SGDID_bis"].isin(feature_name["Primary_SGDID"])]
    edges_list_select.index = range(1, len(edges_list_select) + 1)
    
    return edges_list_select


def distri(genes_list, edges_list, feature_name):
    
    sample_number = 20
    bin_number = 70
    
    edges_list_select = get_edges_list(genes_list, edges_list, feature_name)
    x = list(edges_list_select["3D_distances"])
    H, X1 = np.histogram(x, bins = bin_number)
    F1 = np.cumsum(H)/len(x)
    
    H2 = np.zeros(bin_number)
    rd_x = []
    for i in range(0, sample_number):
        edges_list_random = edges_list.sample(n=len(edges_list_select))
        rd_x_tmp = list(edges_list_random["3D_distances"])
        H_temp, X_temp = np.histogram(rd_x_tmp, bins = bin_number)
        rd_x += rd_x_tmp
        H2 += H_temp
    H2 = H2/sample_number
    F2 = np.cumsum(H2)/sum(H2)
    
    fig, ax = plt.subplots()
    ax.hist(X1[:-1], X1, weights=H2, color="#5767FF", alpha=0.3, label="Random targets")
    ax.set_xlim(0, 200)
    plt.xlabel("3D distances", size = 16)
    ax.hist(x, bins=bin_number, color="#FA3824", alpha=0.3, label="TF's targets")
    plt.ylabel("Count", size = 16)
    ax2=ax.twinx()
    ax2.plot(X1[:-1], F1, label="CDF (TF's targets)", color = "#FA3824")
    ax2.plot(X1[:-1], F2, label="CDF (random)", color = "#5767FF")
    ax.legend(bbox_to_anchor = (0.6, 0.9), loc="upper left")
    ax2.legend(bbox_to_anchor = (0.6, 0.7), loc="upper left")
    
    

    # Compute Kolmogorov-Smirnov test
    # https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ks_2samp.html
    if x != []:
    #if False:
        ks_result = ks_2samp(x, rd_x)
    else:
        ks_result = None
    
    return fig, ks_result

In [ ]:
%%time
adjacency_matrix = pd.read_parquet("../dashboard/static/adjacency_matrix_V4.parquet.gzip", engine='pyarrow')
edges_list = adjacency_matrix.stack().dropna().reset_index()
edges_list.rename(columns = {0: "3D_distances"}, inplace = True)
edges_list = edges_list.sort_values(by="Primary_SGDID")
edges_list

In [ ]:
sql_query = \
"""SELECT Primary_SGDID, Standard_gene_name, Chromosome, Feature_name, Strand, Stop_coordinate, Start_coordinate, Description
FROM SGD_features
"""

# Get all features for all gene
feature_name = tools.get_locus_info("../SCERE.db", sql_query)
feature_name

In [ ]:
#test on one FT
genes_list = pd.read_csv('../results/V2/TF_target_TRN/YPR133C_122_targets.csv', sep=',', header = [0])
files_names = os.listdir("../results/V2/TF_target_TRN")
file_name = 'YPR133C_122_targets.csv'

res = distri(genes_list, edges_list, feature_name)

res[0].savefig(f"../results/V2/distri/{file_name}.png")

In [ ]:
%%time

files_names = os.listdir("../results/V2/TF_target_TRN")
results = pd.DataFrame(columns = ["TF", "TF_name", "description", "targets_number", "KS_stat", "KS_pvalue"])

for file_name in files_names:
    print(file_name)
    FT_name = file_name.split('_')
    FT_name = FT_name[0]
    genes_list = pd.read_csv('../results/V2/TF_target_TRN/' + file_name, sep=',', header = [0])
    res = distri(genes_list, edges_list, feature_name)
    standard_name = feature_name["Standard_gene_name"][feature_name["Feature_name"] == FT_name].values[0]
    description = feature_name["Description"][feature_name["Feature_name"] == FT_name].values[0]
    
    if res[1] != None:
        results = results.append({"TF": FT_name, 
                                  "TF_name": standard_name, 
                                  "description": description, 
                                  "targets_number": len(genes_list), 
                                  "KS_stat": res[1].statistic, 
                                  "KS_pvalue": res[1].pvalue}, 
                                 ignore_index=True)
    
    else:
        results = results.append({"TF": FT_name, 
                                  "TF_name": standard_name, 
                                  "description": description, 
                                  "targets_number": len(genes_list), 
                                  "KS_stat": None, 
                                  "KS_pvalue": None}, 
                                 ignore_index=True)
    
    res[0].savefig(f"../results/V2/distri/{file_name}.png")
    plt.close()

In [ ]:
results

In [ ]:
results.sort_values(by =["KS_stat"])

In [ ]:
fig = px.histogram(results, x="KS_pvalue", nbins = 10)
fig.show()

## 2) For 2D chromosomal positions

In [3]:
sql_query = \
"""SELECT Primary_SGDID, Standard_gene_name, Feature_name, Description, Start_coordinate, Stop_coordinate, Chromosome, Strand
FROM SGD_features
ORDER BY Start_coordinate
"""

# Get all features for all gene
loci = tools.get_locus_info("../SCERE.db", sql_query)
loci

,Primary_SGDID,Standard_gene_name,Feature_name,Description,Start_coordinate,Stop_coordinate,Chromosome,Strand
0,S000028888,,,Terminal telomeric repeats on the left arm of ...,34,1,7,C
1,S000028892,,,Terminal telomeric repeats on the left arm of ...,34,1,8,C
3,S000028919,,,Terminal telomeric repeats on the left arm of ...,51,1,13,C
4,S000001826,,YFL068W,Putative protein of unknown function; SWAT-GFP...,53,535,6,W
5,S000030466,,,,53,535,6,W
...,...,...,...,...,...,...,...,...
16381,S000035945,,,,1525523,1525095,4,C
16382,S000002953,YRF1-1,YDR545W,Helicase encoded by the Y' element of subtelom...,1526321,1531711,4,W
16383,S000036865,,,,1526321,1531711,4,W
16384,S000028617,,YDR545C-A,Dubious open reading frame; unlikely to encode...,1531342,1530863,4,C


In [ ]:
def get_chrom_list(gene_list, loci):
    
    # Add SGDID
    chrom_list = loci[loci["Feature_name"].isin(genes_list["TG"])]
    
    chrom_list.index = range(1, len(chrom_list) + 1)
    
    return chrom_list


def chrom_distri(genes_list, loci):
    
    sample_number = 20
    
    chrom_list = get_chrom_list(genes_list, loci)
    x = list(chrom_list["Chromosome"])
    H, X1 = np.histogram(x, bins = list(range(1, 18)))
    
    H2 = np.zeros(16)
    for i in range(0, sample_number):
        chrom_list_random = loci.sample(n=len(chrom_list))
        rd_x_tmp = list(chrom_list_random["Chromosome"])
        H_temp, X_temp = np.histogram(rd_x_tmp, bins = list(range(1, 18)))
        H2 += H_temp
    H2 = np.round_(H2/sample_number)
    
    fig, ax = plt.subplots()
    ax.hist(X1[:-1], X1, weights=H2, color="#5767FF", alpha=0.3, label="Random targets")
    ax.set_xlim(1, 16)
    plt.xlabel("Chromosomes", size = 16)
    ax.hist(x, bins=list(range(1, 18)), color="#FA3824", alpha=0.3, label="TF's targets")
    plt.ylabel("Count", size = 16)
    ax.legend(bbox_to_anchor = (0.6, 0.9), loc="upper left")

    # Compute chi square test
    #https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chi2_contingency.html    if x != []:
    if ((0 in H) or (0 in H2)) == False:
        chi_test = np.array([H, H2])
        chi_result = chi2_contingency(chi_test)[:2]
    else:
        chi_result = (0, 0)
        
    return fig, chi_result

In [ ]:
genes_list = pd.read_csv('../results/TF_target_TRN/YKR034W_2_targets.csv', sep=',', header = [0])
files_names = os.listdir("../results/TF_target_TRN")
file_name = 'YKR034W_2_targets.csv'

res = chrom_distri(genes_list, loci)

res[0].show()
print(res[1])

In [ ]:
%%time

files_names = os.listdir("../results/TF_target_TRN")
results = pd.DataFrame(columns = ["TF", "TF_name", "description", "targets_number", "Chi_stat", "Chi_pvalue"])

for file_name in files_names:
    print(file_name)
    FT_name = file_name.split('_')
    FT_name = FT_name[0]
    genes_list = pd.read_csv('../results/TF_target_TRN/' + file_name, sep=',', header = [0])
    res = chrom_distri(genes_list, loci)
    standard_name = loci["Standard_gene_name"][loci["Feature_name"] == FT_name].values[0]
    description = loci["Description"][loci["Feature_name"] == FT_name].values[0]
    
    if res[1] != (0, 0):
        results = results.append({"TF": FT_name, 
                                  "TF_name": standard_name, 
                                  "description": description, 
                                  "targets_number": len(genes_list), 
                                  "Chi_stat": res[1][0], 
                                  "Chi_pvalue": res[1][1]}, 
                                 ignore_index=True)
    
    else:
        results = results.append({"TF": FT_name, 
                                  "TF_name": standard_name, 
                                  "description": description, 
                                  "targets_number": len(genes_list), 
                                  "Chi_stat": None, 
                                  "Chi_pvalue": None}, 
                                 ignore_index=True)
    
    res[0].savefig(f"../results/chrom_distri/{file_name}.png")
    plt.close()

In [ ]:
results.sort_values(by =["Chi_stat"], ascending=False)

In [ ]:
query = """
SELECT Primary_SGDID, Feature_name, Feature_type, Start_coordinate, Strand, Chromosome
FROM SGD_features
"""

cursor = db_connexion.cursor()
    
# Query database.
chrom_info = cursor.execute(query)
    
# Convert to Pandas dataframe
column_names = [column[0] for column in chrom_info.description]
chrom_info_df = pd.DataFrame(chrom_info.fetchall(), columns=column_names)
    
# Select only strands + and -
chrom_info_df = chrom_info_df[ (chrom_info_df["Strand"] == "C") | (chrom_info_df["Strand"] == "W") ]
# Remove "2-micron" plasmid
chrom_info_df = chrom_info_df[ chrom_info_df["Chromosome"] != "2-micron" ]
# Convert chromosome id to int
chrom_info_df["Chromosome"] = chrom_info_df["Chromosome"].astype(int)

In [ ]:
chrom_info_df[ chrom_info_df["Feature_type"] != "CDS" ]

In [12]:
Yeast_TRN = pd.read_csv('../results/yeast2019-full-conds-net.csv', sep='\t', header = None)
Yeast_TRN.columns = ["TF", "TG", "type", "description"]
Yeast_TRN

,TF,TG,type,description
0,ABF1,COX6,Direct Positive,N/A;N/A;N/A;Stress;Unstressed log-phase growth...
1,ABF1,RPS28B,Direct,N/A;N/A;Stress
2,ABF1,RPO26,Direct,N/A;N/A;N/A;Unstressed log-phase growth (control)
3,ABF1,HIS7,Direct Dual,N/A;In vitro;Unstressed log-phase growth (cont...
4,ABF1,"ADE5,7",Direct,N/A;Unstressed log-phase growth (control)
...,...,...,...,...
195493,PEX2,PEX2,Indirect Negative,Unstressed log-phase growth (control);Stress
195494,FYV5,ATG14,Indirect Negative,Unstressed log-phase growth (control)
195495,FYV5,ATG9,Indirect Negative,Unstressed log-phase growth (control)
195496,FYV5,ATG32,Indirect Negative,Unstressed log-phase growth (control)


In [9]:
len(Yeast_TRN[1].unique())

6886

In [13]:
selected_Yeast_TRN = Yeast_TRN[Yeast_TRN.type.isin(['Direct Positive', 'Direct', 'Direct Dual', 'Direct Negative'])]

In [16]:
len(selected_Yeast_TRN.TG.unique())

6475